In [46]:
import tkinter as tk
from tkinter import ttk

# Simulated database (replace with SQLite in final version)
booked_seats_by_date = {
    "08\nSUN": [("A", 2), ("B", 4), ("C", 3)],
    "09\nMON": [("D", 2), ("E", 4), ("F", 3)],
    "10\nTUE": [("G", 2), ("H", 4), ("I", 3)],
    "11\nWED": [("J", 2), ("A", 4), ("B", 3)],
    "12\nTHU": [("C", 2), ("D", 4), ("E", 3)],  # Current date: June 12th, 06:06 PM WAT
    "13\nFRI": [("F", 2), ("G", 4), ("H", 3)],
    "14\nSAT": [("I", 2), ("J", 4), ("A", 3)]
}
selected_date = None
selected_time = None
selected_seats = []

# Global widget references
canvas = None
scrollbar = None
inner_frame = None
time_frame = None
movie_frame = None
movie_card = None
viewing_date = None
order_frame = None
seat_frame = None
checkout_frame = None
ticket_frame = None
step_frame = None
title_label = None
checkout_btn = None
date_cancel_btn = None  # Cancel button in dates area
movie_cancel_btn = None  # Cancel button in movie card

def select_date(date):
    global selected_date
    selected_date = date
    for btn in date_buttons:
        btn.pack_forget()  # Hide date buttons when a date is selected
    sender = date_buttons[dates.index(date)]
    sender.config(bg="yellow", relief="raised", fg="lightgray")
    update_times()
    update_seat_colors()  # Update seat colors based on booked seats for the selected date
    update_movie_card()  # Refresh movie card
    update_date_cancel_btn()  # Display and update date cancel button

def update_times():
    global time_frame
    if time_frame:
        for widget in time_frame.winfo_children():
            if widget.widgetName == "button" and widget["text"] not in ["Viewing Data"]:
                widget.destroy()
        times = {
            "08\nSUN": ["08:00 AM", "09:40 AM", "11:40 AM"],
            "09\nMON": ["10:00 AM", "12:00 PM", "02:00 PM"],
            "10\nTUE": ["09:00 AM", "11:00 AM", "01:00 PM"],
            "11\nWED": ["08:30 AM", "10:30 AM", "12:30 PM"],
            "12\nTHU": ["07:00 PM", "09:00 PM", "11:00 PM"],  # Updated for current time: 06:06 PM WAT
            "13\nFRI": ["08:00 AM", "10:00 AM", "12:00 PM"],
            "14\nSAT": ["09:30 AM", "11:30 AM", "01:30 PM"]
        }
        if selected_date:
            for time in times.get(selected_date, ["07:00 PM", "09:00 PM", "11:00 PM"]):
                btn = tk.Button(time_frame, text=time, fg="lightgray", bg="gray", width=10, relief="flat",
                                activebackground="darkgray", command=lambda t=time: select_time(t))
                btn.pack(pady=5)
                if time == selected_time:
                    btn.config(bg="yellow", relief="raised", fg="lightgray")
        else:
            for btn in date_buttons:
                btn.pack(side=tk.LEFT, padx=5)
                btn.config(bg="gray", relief="flat", fg="lightgray")
            update_date_cancel_btn()  # Ensure cancel button state is updated

def select_time(time):
    global selected_time
    selected_time = time
    for btn in time_frame.winfo_children():
        if btn.widgetName == "button" and btn["text"] not in ["Viewing Data"]:
            btn.config(bg="gray", relief="flat", fg="lightgray")
    sender = [b for b in time_frame.winfo_children() if b["text"] == time][0]
    sender.config(bg="yellow", relief="raised", fg="lightgray")
    update_movie_card()

def update_movie_card():
    global movie_card, viewing_date, movie_cancel_btn
    if movie_card:
        for widget in movie_card.winfo_children():
            widget.destroy()
        movie_label = tk.Label(movie_card, text="MISSION IMPOSSIBLE: FINAL RECKONING", font=("Arial", 14, "bold"), fg="yellow", bg="black")
        movie_label.pack(pady=2)
        details = tk.Label(movie_card, text="2025, NA, 150MIN", fg="white", bg="black")
        details.pack()
        if selected_date:
            viewing_date = tk.Button(movie_card, text=f"Thu(June 12th)\n{selected_time}" if selected_time else f"Thu(June 12th)\n07:00 PM", fg="lightgray", bg="yellow", width=15, relief="flat",
                                    activebackground="yellow", command=lambda: print("Viewing Date Selected"))
            viewing_date.pack(pady=5)
        else:
            viewing_date = tk.Button(movie_card, text="Thu(June 12th)\n07:00 PM", fg="lightgray", bg="yellow", width=15, relief="flat",
                                    activebackground="yellow", command=lambda: print("Viewing Date Selected"))
            viewing_date.pack(pady=5)
        # Add movie cancel button (updates command based on selected_date)
        if movie_cancel_btn:
            movie_cancel_btn.destroy()
        movie_cancel_btn = tk.Button(movie_card, text="Cancel", fg="lightgray", bg="gray", width=15, relief="flat",
                                    command=opt_out_date if selected_date else lambda: None)
        movie_cancel_btn.pack(pady=5)

def opt_out_date():
    global selected_date, selected_time, selected_seats
    selected_date = None
    selected_time = None
    selected_seats.clear()
    update_movie_card()
    update_seat_colors()
    update_times()  # Re-display date buttons
    time_frame.update()  # Force UI update to ensure changes are applied
    update_date_cancel_btn()  # Update date cancel button state

def update_date_cancel_btn():
    global date_cancel_btn
    if date_cancel_btn:
        date_cancel_btn.destroy()
    if selected_date:  # Only create and show cancel button when a date is selected
        date_cancel_btn = tk.Button(time_frame, text="Cancel", fg="lightgray", bg="gray", width=5, height=2, relief="flat",
                                    activebackground="darkgray", command=opt_out_date, state="normal")
        date_cancel_btn.pack(side=tk.LEFT, padx=5)

def update_seat_colors():
    global seat_frame, seats
    if seat_frame and seats:
        current_booked_seats = booked_seats_by_date.get(selected_date, [])
        for row in range(10):  # Rows A-J
            for col in range(2, 6):  # Columns 2-5
                seat_coord = (chr(65 + row), col)  # A=65 in ASCII
                if seat_coord in seats:
                    if seat_coord in current_booked_seats:
                        seats[seat_coord].config(bg="green")  # Green for booked seats
                    elif seat_coord in selected_seats:
                        seats[seat_coord].config(bg="green")  # Green for selected seats
                    else:
                        seats[seat_coord].config(bg="gray")  # Gray for available seats

def select_seat(seat_coord):
    if seat_coord not in booked_seats_by_date.get(selected_date, []):  # Only allow selection if not booked
        if seat_coord in selected_seats:
            selected_seats.remove(seat_coord)
            seats[seat_coord].config(bg="gray")
        else:
            selected_seats.append(seat_coord)
            seats[seat_coord].config(bg="green")
    update_total()
    update_seat_colors()  # Ensure colors update after selection

def update_total():
    if order_frame and total_label:
        total_label.config(text=f"Total: N{len(selected_seats) * 5000:.2f}")

def show_checkout():
    global checkout_frame
    hide_all_frames()
    if not checkout_frame:
        checkout_frame = tk.Frame(inner_frame, bg="black")
        checkout_label = tk.Label(checkout_frame, text="CHECKOUT", font=("Arial", 20, "bold"), fg="yellow", bg="black")
        checkout_label.pack(pady=10)
        details_label = tk.Label(checkout_frame, text=f"Movie: MISSION IMPOSSIBLE: FINAL RECKONING\nDate: {selected_date}\nTime: {selected_time}\nSeats: {', '.join(map(str, selected_seats))}\nTotal: N{len(selected_seats) * 5000:.2f}", fg="white", bg="black")
        details_label.pack(pady=10)
        confirm_btn = tk.Button(checkout_frame, text="Confirm Payment", fg="lightgray", bg="yellow", width=15, relief="flat",
                               command=show_ticket_purchased)
        confirm_btn.pack(pady=10)
    checkout_frame.pack(pady=10)

def show_ticket_purchased():
    global ticket_frame
    hide_all_frames()
    if not ticket_frame:
        ticket_frame = tk.Frame(inner_frame, bg="black")
        ticket_label = tk.Label(ticket_frame, text="TICKET PURCHASED", font=("Arial", 20, "bold"), fg="yellow", bg="black")
        ticket_label.pack(pady=10)
        details_label = tk.Label(ticket_frame, text=f"Movie: MISSION IMPOSSIBLE: FINAL RECKONING\nDate: {selected_date}\nTime: {selected_time}\nSeats: {', '.join(map(str, selected_seats))}\nTotal: N{len(selected_seats) * 5000:.2f}", fg="white", bg="black")
        details_label.pack(pady=10)
        back_btn = tk.Button(ticket_frame, text="Back to Booking", fg="lightgray", bg="gray", width=15, relief="flat",
                             command=setup_screen)
        back_btn.pack(pady=10)
    ticket_frame.pack(pady=10)
    # Update booked seats after purchase
    if selected_date and selected_seats:
        booked_seats_by_date[selected_date] = list(set(booked_seats_by_date.get(selected_date, []) + selected_seats))
        selected_seats.clear()

def hide_all_frames():
    if time_frame and time_frame.winfo_manager() == 'pack':
        time_frame.pack_forget()
    if movie_frame and movie_frame.winfo_manager() == 'pack':
        movie_frame.pack_forget()
    if order_frame and order_frame.winfo_manager() == 'pack':
        order_frame.pack_forget()
    if seat_frame and seat_frame.winfo_manager() == 'pack':
        seat_frame.pack_forget()
    if checkout_frame and checkout_frame.winfo_manager() == 'pack':
        checkout_frame.pack_forget()
    if ticket_frame and ticket_frame.winfo_manager() == 'pack':
        ticket_frame.pack_forget()
    if step_frame and step_frame.winfo_manager() == 'pack':
        step_frame.pack_forget()
    if title_label and title_label.winfo_manager() == 'pack':
        title_label.pack_forget()
    if checkout_btn and checkout_btn.winfo_manager() == 'pack':
        checkout_btn.pack_forget()

def setup_screen():
    global canvas, scrollbar, inner_frame, time_frame, movie_frame, movie_card, viewing_date, order_frame, seat_frame, date_buttons, seats, total_label, step_frame, title_label, checkout_btn, date_cancel_btn, movie_cancel_btn
    hide_all_frames()

    # Canvas and Scrollbar
    if not canvas:
        canvas = tk.Canvas(root, bg="black")
        scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)
        canvas.configure(yscrollcommand=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        inner_frame = tk.Frame(canvas, bg="black")
        canvas.create_window((0, 0), window=inner_frame, anchor="nw")
        inner_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

    # Step Indicators
    step_frame = tk.Frame(inner_frame, bg="black")
    step_frame.pack(pady=10)
    steps = ["1", "2", "3", "4"]
    for i, step in enumerate(steps):
        lbl = tk.Label(step_frame, text=step, bg="gray", fg="white", width=5, height=2, relief="flat")
        lbl.pack(side=tk.LEFT, padx=5)
        if i == 0:
            lbl.config(bg="yellow", relief="raised", fg="lightgray")

    # Title
    title_label = tk.Label(inner_frame, text="BOOK A TICKET", font=("Arial", 20, "bold"), fg="yellow", bg="black")
    title_label.pack(pady=10)

    # Showing Time Section
    time_frame = tk.LabelFrame(inner_frame, text="SHOWING TIME", fg="yellow", bg="black", font=("Arial", 12, "bold"), padx=10, pady=5)
    time_frame.pack(pady=20, padx=20, fill="x")
    viewing_data = tk.Label(time_frame, text="Viewing Data", fg="white", bg="black", font=("Arial", 10))
    viewing_data.pack()

    # Date buttons
    dates = ["08\nSUN", "09\nMON", "10\nTUE", "11\nWED", "12\nTHU", "13\nFRI", "14\nSAT"]
    date_buttons = []
    for date in dates:
        btn = tk.Button(time_frame, text=date, fg="lightgray", bg="gray", width=5, height=2, relief="flat",
                        activebackground="darkgray", command=lambda d=date: select_date(d))
        btn.pack(side=tk.LEFT, padx=5)
        date_buttons.append(btn)
    if selected_date:
        date_buttons[dates.index(selected_date)].config(bg="yellow", relief="raised", fg="lightgray")

    # Initial time buttons (only if no date selected initially)
    if not selected_date:
        times = ["07:00 PM", "09:00 PM", "11:00 PM"]  # Updated for current time: 06:06 PM WAT
        for time in times:
            btn = tk.Button(time_frame, text=time, fg="lightgray", bg="gray", width=10, relief="flat",
                            activebackground="darkgray", command=lambda t=time: select_time(t))
            btn.pack(pady=5)

    # Movie Details
    movie_frame = tk.Frame(inner_frame, bg="black")
    movie_frame.pack(pady=10, padx=10, fill="x")
    movie_card = tk.Frame(movie_frame, bg="black", bd=2, relief="raised")
    movie_card.pack(pady=5, padx=5, fill="x")
    movie_label = tk.Label(movie_card, text="MISSION IMPOSSIBLE: FINAL RECKONING", font=("Arial", 14, "bold"), fg="yellow", bg="black")
    movie_label.pack(pady=2)
    details = tk.Label(movie_card, text="2025, NA, 150MIN", fg="white", bg="black")
    details.pack()
    viewing_date = tk.Button(movie_card, text="Thu(June 12th)\n07:00 PM", fg="lightgray", bg="yellow", width=15, relief="flat",
                             activebackground="yellow", command=lambda: print("Viewing Date Selected"))
    viewing_date.pack(pady=5)
    # Add movie cancel button
    movie_cancel_btn = tk.Button(movie_card, text="Cancel", fg="lightgray", bg="gray", width=15, relief="flat",
                                 command=opt_out_date if selected_date else lambda: None)
    movie_cancel_btn.pack(pady=5)

    # Order Details
    order_frame = tk.LabelFrame(inner_frame, text="Order Details", fg="yellow", bg="black", font=("Arial", 12, "bold"), padx=10, pady=5)
    order_frame.pack(pady=10, padx=10, fill="x")
    total_label = tk.Label(order_frame, text="Total: N0.00", fg="white", bg="black")
    total_label.pack()

    # Seat Selection (Movie Theater Layout)
    seat_frame = tk.LabelFrame(inner_frame, text="SELECT SEATS", fg="yellow", bg="black", font=("Arial", 12, "bold"), padx=10, pady=5)
    seat_frame.pack(pady=10, padx=10, fill="x")
    seats = {}
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    for i, row in enumerate(rows):
        for j in range(2, 6):  # Columns 2 to 5 as per image
            seat_coord = (row, j)
            btn = tk.Button(seat_frame, text=f"{row}{j}", fg="white", bg="gray",
                            width=5, height=2, relief="flat", activebackground="darkgray",
                            command=lambda coord=seat_coord: select_seat(coord))
            btn.grid(row=i, column=j-2, padx=2, pady=2)  # Adjust column index to start at 0
            seats[seat_coord] = btn
    update_seat_colors()  # Initial seat color update

    # Checkout Button
    checkout_btn = tk.Button(inner_frame, text="Checkout", fg="lightgray", bg="yellow", width=15, relief="flat",
                             command=show_checkout)
    checkout_btn.pack(pady=10)

# Initialize the application
root = tk.Tk()
root.title("Book a Ticket")
root.configure(bg="black")
root.geometry("800x600")
setup_screen()
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ISMS\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\ISMS\AppData\Local\Temp\ipykernel_2984\561336634.py", line 117, in opt_out_date
    update_times()  # Re-display date buttons
    ~~~~~~~~~~~~^^
  File "C:\Users\ISMS\AppData\Local\Temp\ipykernel_2984\561336634.py", line 72, in update_times
    btn.pack(side=tk.LEFT, padx=5)
    ~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISMS\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2596, in pack_configure
    self.tk.call(
    ~~~~~~~~~~~~^
          ('pack', 'configure', self._w)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
          + self._options(cnf, kw))
          ^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: bad window path name ".!canvas.!frame.!labelframe.!button"
Exception in Tkinter callback
Traceback (most recent ca